# ML_day1_Titanic

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from lightgbm import LGBMClassifier
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import KFold
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Read data

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

## Show more data information

In [ ]:
print('Part of missing values for train data')
print(train_data.isnull().sum() / len(train_data))
print('Part of missing values for test data')
print(test_data.isnull().sum() / len(test_data))
# Because there are lots of Na values in  Age column and Cabin these column could not be features

# Although Embarked value have Na in train data, the test data is clean.
# The fare is clean in Train and have a little NA in test_data
# I delete the Na row in train data and use embarked as a feature to predict
print(type(train_data))
train_data = train_data.dropna(subset=["Embarked"])
test_data["Fare"] = test_data["Fare"].fillna(test_data['Fare'].mean())


In [ ]:
# Check the NA
print('Part of missing values for train data')
print(train_data.isnull().sum() / len(train_data))
print('Part of missing values for test data')
print(test_data.isnull().sum() / len(test_data))

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

In [ ]:
# Try to fix the Age feature
# In this part, I think that men and women will have different reason not to fill in the age information, so I take a closer look for each gender.
agena = train_data.loc[train_data['Age'].isna()]
agmen = agena.loc[agena.Sex == 'male']
agwomen = agena.loc[agena.Sex == 'female']
print(len(agmen))
print(len(agwomen))
# Surprisingly more man's age is a secret.

In [ ]:
# At the begging I just want to fill Age by mean value
#train_data["Age"] = train_data["Age"].fillna(train_data['Age'].mean())
#test_data["Age"] = test_data["Age"].fillna(test_data['Age'].mean())

#Now, I want to print out the mean age value of each gender and fill them in to the NA
print(train_data.loc[train_data.Sex == 'male'].mean())
print(train_data.loc[train_data.Sex == 'female'].mean())
#Use group by to fill na by each gender
train_data["Age"] = train_data.groupby(['Sex'],sort=False)["Age"].fillna(train_data['Age'].mean())
test_data["Age"] = test_data.groupby(['Sex'],sort=False)["Age"].fillna(train_data['Age'].mean())
#Double check does it works
print(train_data.loc[train_data.Sex == 'male'].mean())
print(train_data.loc[train_data.Sex == 'female'].mean())

In [ ]:
# Check the NA
print('Part of missing values for train data')
print(train_data.isnull().sum() / len(train_data))
print('Part of missing values for test data')
print(test_data.isnull().sum() / len(test_data))

## Model training

In [ ]:
features = ["Pclass", "Sex", "SibSp", "Parch","Embarked", "Fare","Age"]
X = pd.get_dummies(train_data[features])
y = train_data["Survived"]
X_test = pd.get_dummies(test_data[features])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions_rfc = model.predict(X_test)

predictions=predictions_rfc

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions })
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")